In [50]:
import pandas as pd

In [134]:
from neo4j import GraphDatabase

class Neo4jQueries(object):
    def __init__(self,uri,usr,password):
        self._driver = GraphDatabase.driver(uri, auth=(usr, password), encrypted =False)

    def close(self):
        self._driver.close()
        
    def prepare_data(self,path_file):
        self.df = df_habitant = pd.read_excel(path_file)
        self.df_clean = self.df[['Dp','Commune','Habitants','Densité']].copy()
        self.df_clean = self.df_clean.rename(columns={'Densité':'densite'})
        
    def run_query(self,method,*args):
        
        with self._driver.session() as session:
            if(args):
                return session.write_transaction(method,args[0])
            else:
                return session.write_transaction(method)
    def _create_and_return_db(self,tx):
        create_db_query = """CREATE (c:Country {Dp: toInteger($dp), Commune: $commune, Habitant: $habitants, densite: $densite})"""
        for index, row in self.df_clean.iterrows():
            result = tx.run(create_db_query,parameters = {'dp':row['Dp'], 'commune':row['Commune'],'habitants':row['Habitants'],'densite':row['densite']})
        return result
    
    def create_relations(self,tx):
        query = """match(c:Country {name:c.Dp})
                   match(b:Country {name:b.Commune})
                   MERGE (c)-[:HAVE]->(b)
                """
        result = tx.run(query)
        return result

    # first question
    def display_municpalities(self,tx):
        query =  """ MATCH (c:Country)
                     RETURN c.Commune """
        result = tx.run(query)
        return result
    
    #seconde question
    def display_commune_given_dp(self,tx,dp):
        query = """ MATCH (c:Country) 
                    WHERE c.Dp = $dp
                    RETURN c.Commune
                """
        result = tx.run(query,parameters= {'dp':dp})
        return result
    
    #third question
    def info_most_populated(self,tx):
        query = """ MATCH(c:Country)
                    RETURN c.Dp, sum(c.Habitant), max(c.Habitant)
                """
        result = tx.run(query)
        return result
create = Neo4jQueries('bolt://localhost:7687','neo4j','Lazaret123*')
create.prepare_data('Data/habitants90.xls')


# Seconde Question

In [135]:
departement = 94

s_result = create.run_query(create.display_commune_given_dp,departement)
print('toute les communes du: ', departement,'\n')
for q in s_result:
    print(q['c.Commune'])


toute les communes du:  94 

Ablon-sur-Seine
Alfortville
Arcueil
Boissy-St-Léger
Bonneuil-sur-Marne
Bry-sur-Marne
Cachan
Champigny-sur-Marne
Charenton-le-Pont
Chennevières-sur-Marne
Chevilly-Larue
Choisy-le-Roi
Créteil
Fontenay-sous-Bois
Fresnes
Gentilly
Haye-les-Roses 
Ivry-sur-Seine
Joinville-le-Pont
Kremlin-Bicêtre 
Limeil-Brévannes
Maisons-Alfort
Mandres-les-Roses
Marolles-en-Brie
Nogent-sur-Marne
Noiseau
Orly
Ormesson-sur-Marne
Périgny
Perreux-sur-Marne 
Plessis-Trévise 
Queue-en-Brie 
Rungis
St-Mandé
St-Maur-des-Fossés
St-Maurice
Santeny
Sucy-en-Brie
Thiais
Valenton
Villecresnes
Villejuif
Villeneuve-le-Roi
Villeneuve-St-Georges
Villiers-sur-Marne
Vincennes
Vitry-sur-Seine


# Third question

In [137]:
t_result = create.run_query(create.info_most_populated)
print('Departement','Sum(Habitants), Max(Habitants)')
for q in t_result:
    print(q['c.Dp'],',',q['sum(c.Habitant)'],',',q['max(c.Habitant)'])

Departement Sum(Habitants), Max(Habitants)
1 , 471019 , 40972
2 , 537259 , 60644
3 , 357710 , 44248
4 , 130883 , 19107
5 , 113300 , 33444
6 , 971829 , 342439
7 , 277581 , 18525
8 , 296357 , 57008
9 , 136455 , 12965
10 , 269053 , 59255
11 , 298712 , 45849
12 , 270141 , 24701
13 , 1759371 , 800550
14 , 618478 , 112846
15 , 158723 , 30773
16 , 341993 , 42876
17 , 527146 , 71094
18 , 321559 , 75609
19 , 237908 , 49765
21 , 493866 , 146703
22 , 538395 , 44752
23 , 131349 , 14706
24 , 386365 , 30280
25 , 484770 , 113828
26 , 414072 , 63437
27 , 513818 , 49103
28 , 396073 , 39595
29 , 838687 , 147956
None , 250371 , 58949
30 , 585049 , 128471
31 , 925962 , 358688
32 , 174587 , 23136
33 , 1213499 , 210336
34 , 794603 , 207996
35 , 798718 , 197536
36 , 237510 , 50969
37 , 529345 , 129509
38 , 1016228 , 150758
39 , 248759 , 26577
40 , 311461 , 28328
41 , 305937 , 49318
42 , 746288 , 199396
43 , 206568 , 21743
44 , 1052183 , 244995
45 , 580612 , 105111
46 , 155816 , 19735
47 , 305989 , 30553
48 ,

# First Question

In [138]:
f_result = create.run_query(create.display_municpalities)
print('toutes les communes: \n')
for q in f_result:
    print(q['c.Commune'])


toutes les communes: 

Abergement-Clémenciat 
Abergement-de-Varey 
Ambérieu-en-Bugey
Ambérieux-en-Dombes
Ambléon
Ambronay
Ambutrix
Andert-et-Condon
Anglefort
Apremont
Aranc
Arandas
Arbent
Arbignieu
Arbigny
Argis
Armix
Ars-sur-Formans
Artemare
Asnières-sur-Saône
Attignat
Bâgé-la-Ville
Bâgé-le-Châtel
Balan
Baneins
Beaupont
Beauregard
Bellignat
Béligneux
Bellegarde-sur-Valserine
Belley
Belleydoux
Belmont-Luthézieu
Bénonces
Bény
Béon
Béréziat
Bettant
Bey
Beynost
Billiat
Birieux
Biziat
Blyes
Boisse 
Boissey
Bolozon
Bouligneux
Bourg-en-Bresse
Bourg-St-Christophe
Boyeux-St-Jérôme
Boz
Brégnier-Cordon
Brénaz
Brénod
Brens
Bressolles
Brion
Briord
Buellas
Burbanche 
Ceignes
Cerdon
Certines
Cessy
Ceyzériat
Ceyzérieu
Chalamont
Chaleins
Chaley
Challes
Challex
Champagne-en-Valromey
Champdor
Champfromier
Chanay
Chaneins
Chanoz-Châtenay
Chapelle-du-Châtelard 
Charix
Charnoz-sur-Ain
Château-Gaillard
Châtenay
Châtillon-en-Michaille
Châtillon-la-Palud
Châtillon-sur-Chalaronne
Chavannes-sur-Reyssouze
Chavan

Savignac-les-Ormeaux
Ségura
Seix
Sem
Senconac
Lorp-Sentaraille
Sentein
Sentenac-d'Oust
Sentenac-de-Sérou
Serres-sur-Arget
Sieuras
Siguer
Sinsat
Sor
Sorgeat
Soueix-Rogalle
Soula
Soulan
Suc-et-Sentenac
Surba
Suzan
Tabre
Tarascon-sur-Ariège
Taurignan-Castet
Taurignan-Vieux
Teilhet
Thouars-sur-Arize
Tignac
Tour-du-Crieu 
Tourtouse
Tourtrol
Trémoulet
Troye-d'Ariège
Uchentein
Unac
Unzent
Urs
Ussat
Ustou
Vals
Varilhes
Vaychis
Vèbre
Ventenac
Verdun
Vernajoul
Vernaux
Vernet 
Verniolle
Vicdessos
Villeneuve
Villeneuve-d'Olmes
Villeneuve-du-Latou
Villeneuve-du-Paréage
Vira
Viviès
Ste-Suzanne
Ailleville
Aix-en-Othe
Allibaudières
Amance
Arcis-sur-Aube
Arconville
Argançon
Arrelles
Arrembécourt
Arrentières
Arsonval
Assenay
Assencières
Aubeterre
Aulnay
Auxon
Val-d'Auzon
Avant-lès-Marcilly
Avant-lès-Ramerupt
Avirey-Lingey
Avon-la-Pèze
Avreuil
Bagneux-la-Fosse
Bailly-le-Franc
Balignicourt
Balnot-la-Grange
Balnot-sur-Laignes
Barberey-St-Sulpice
Barbuise
Baroville
Bar-sur-Aube
Bar-sur-Seine
Bayel
Bercenay-

Médis
Mérignac
Meschers-sur-Gironde
Messac
Meursac
Meux
Migré
Migron
Mirambeau
Moëze
Moings
Mons
Montendre
Montguyon
Montils
Montlieu-la-Garde
Montpellier-de-Médillan
Montroy
Moragne
Mornac-sur-Seudre
Mortagne-sur-Gironde
Mortiers
Mosnac
Mung 
Muron
Nachamps
Nancras
Nantillé
Néré
Neuillac
Neulles
Neuvicq
Neuvicq-le-Château
Nieul-lès-Stes
Nieul-le-Virouil
Nieul-sur-Mer
Nieulle-sur-Seudre
Nouillers 
Nuaillé-d'Aunis
Nuaillé-sur-Boutonne
Orignolles
Ozillac
Paillé
Péré
Pérignac
Périgny
Pessines
Pin 
St-Denis-du-Pin
Pisany
Plassac
Plassay
Polignac
Pommiers-Moulons
Pons
Pont-l'Abbé-d'Arnoult
Port-d'Envaux
Portes-en-Ré 
Pouillac
Poursay-Garnaud
Préguillac
Prignac
Puilboreau
Puy-du-Lac
Puyravault
Puyrolland
Réaux
Rétaud
Rivedoux-Plage
Rioux
Rochefort
Rochelle 
Romazières
Romegoux
Ronde 
Rouffiac
Rouffignac
Royan
Sablonceaux
St-Agnant
St-Aigulin
St-André-de-Lidon
St-Augustin
St-Bonnet-sur-Gironde
St-Bris-des-Bois
St-Césaire
St-Christophe
St-Ciers-Champagne
St-Ciers-du-Taillon
St-Clément-des-Bale

Grand-Bourg 
Guéret
Issoudun-Létrieix
Jalesches
Janaillat
Jarnages
Jouillat
Ladapeyre
Lafat
Lavaufranche
Lavaveix-les-Mines
Lépaud
Lépinas
Leyrat
Linard
Lioux-les-Monges
Lizières
Lourdoueix-St-Pierre
Lupersat
Lussat
Magnat-l'Etrange
Mainsat
Maison-Feyne
Maisonnisses
Malleret
Malleret-Boussac
Malval
Mansat-la-Courrière
Mars 
Marsac
Mas-d'Artige 
Masbaraud-Mérignat
Mautes
Mazeirat
Mazière-aux-Bons-Hommes 
Méasnes
Mérinchal
Montaigut-le-Blanc
Montboucher
Monteil-au-Vicomte 
Mortroux
Mourioux-Vieilleville
Moutier-d'Ahun
Moutier-Malcard
Moutier-Rozeille
Naillat
Néoux
Noth
Nouaille 
Nouhant
Nouzerines
Nouzerolles
Nouziers
Parsac
Peyrabout
Peyrat-la-Nonière
Pierrefitte
Pionnat
Pontarion
Pontcharraud
Pouge 
Poussanges
Puy-Malsignat
Reterre
Rimondeix
Roches
Rougnat
Royère-de-Vassivière
Sagnat
Sannat
Sardent
Saunière 
Savennes
Sermur
Serre-Bussière-Vieille 
Soubrebost
Soumans
Sous-Parsat
Souterraine 
St-Agnant-de-Versillat
St-Agnant-près-Crocq
St-Alpinien
St-Amand
St-Amand-Jartoudeix
St-Avit-de-

Congénies
Connaux
Conqueyrac
Corbès
Corconne
Cornillon
Courry
Crespian
Cros
Cruviers-Lascours
Deaux
Dions
Domazan
Domessargues
Dourbies
Durfort-et-St-Martin-de-Sossena
Estézargues
Estréchure 
Euzet
Flaux
Foissac
Fons
Fons-sur-Lussan
Fontanès
Fontarèches
Fournès
Fourques
Fressac
Gagnières
Gailhan
Gajan
Gallargues-le-Montueux
Garn 
Garons
Garrigues-Ste-Eulalie
Gaujac
Générac
Générargues
Génolhac
Goudargues
Grand-Combe 
Grau-du-Roi 
Issirac
Jonquières-St-Vincent
Junas
Lamelouze
Langlade
Lanuéjols
Lasalle
Laudun
Laval-Pradel
Laval-St-Roman
Lecques
Lédenon
Lédignan
Lézan
Liouc
Lirac
Logrian-Florian
Lussan
Mages 
Malons-et-Elze
Mandagout
Manduel
Marguerittes
Mars
Martignargues
Martinet 
Maruéjols-lès-Gardon
Massanes
Massillargues-Attuech
Mauressargues
Méjannes-le-Clap
Méjannes-lès-Alès
Meynes
Meyrannes
Mialet
Milhaud
Molières-Cavaillac
Molières-sur-Cèze
Monoblet
Mons
Montaren-et-St-Médiers
Montclus
Montdardier
Monteils
Montfaucon
Montfrin
Montignargues
Montmirat
Montpezat
Moulézan
Moussac
Mu

...

Varennes-lès-Narcy
Varennes-Vauzelles
Varzy
Vauclaix
Verneuil
Vielmanay
Vignol
Villapourçon
Villiers-le-Sec
Ville-Langy
Villiers-sur-Yonne
Vitry-Laché
Abancourt
Abscon
Aibes
Aix
Allennes-les-Marais
Amfroipret
Anhiers
Aniche
Villeneuve-d'Ascq
Anneux
Annoeullin
Anor
Anstaing
Anzin
Arleux
Armbouts-Cappel
Armentières
Arnèke
Artres
Assevent
Attiches
Aubencheul-au-Bac
Auberchicourt
Aubers
Aubigny-au-Bac
Aubry-du-Hainaut
Auby
Auchy-lez-Orchies
Audignies
Aulnoy-lez-Valenciennes
Aulnoye-Aymeries
Avelin
Avesnelles
Avesnes-sur-Helpe
Avesnes-les-Aubert
Avesnes-le-Sec
Awoingt
Bachant
Bachy
Bailleul
Baisieux
Baives
Bambecque
Banteux
Bantigny
Bantouzelle
Bas-Lieu
Bassée 
Bauvin
Bavay
Bavinchove
Bazuel
Beaucamps-Ligny
Beaudignies
Beaufort
Beaumont-en-Cambrésis
Beaurain
Beaurepaire-sur-Sambre
Beaurieux
Beauvois-en-Cambrésis
Bellaing
Bellignies
Bérelles
Bergues
Berlaimont
Bermerain
Bermeries
Bersée
Bersillies
Berthen
Bertry
Béthencourt
Bettignies
Bettrechies
Beugnies
Beuvrages
Beuvry-la-Forêt
Bévillers


Thièvres
Tigny-Noyelle
Tilloy-lès-Hermaville
Tilloy-lès-Mofflaines
Tilly-Capelle
Tilques
Tincques
Tingry
Tollent
Torcy
Tortefontaine
Tortequesne
Touquet-Paris-Plage 
Tournehem-sur-la-Hem
Tramecourt
Transloy 
Trescault
Troisvaux
Tubersent
Vacquerie-le-Boucq
Vacqueriette-Erquières
Valhuon
Vaudricourt
Vaudringhem
Vaulx
Vaulx-Vraucourt
Vélu
Vendin-lès-Béthune
Vendin-le-Vieil
Verchin
Verchocq
Verlincthun
Vermelles
Verquigneul
Verquin
Verton
Vieil-Hesdin
Vieille-Chapelle
Vieille-Eglise
Vieil-Moutier
Villers-au-Bois
Villers-au-Flos
Villers-Brûlin
Villers-Châtel
Villers-lès-Cagnicourt
Villers-l'Hôpital
Villers-Sir-Simon
Vimy
Vincly
Violaines
Vis-en-Artois
Vitry-en-Artois
Waben
Wacquinghen
Wail
Wailly
Wailly-Beaucamp
Wambercourt
Wamin
Wancourt
Wanquetin
Wardrecques
Warlencourt-Eaucourt
Warlincourt-lès-Pas
Warlus
Warluzel
Wast 
Beauvoir-Wavans
Wavrans-sur-l'Aa
Wavrans-sur-Ternoise
Westrehem
Wicquinghem
Widehem
Wierre-au-Bois
Wierre-Effroy
Willeman
Willencourt
Willerval
Wimereux
Wimille
Wingles
W

Raze
Recologne
Recologne-lès-Rioz
Renaucourt
Grande-Résie 
Résie-St-Martin 
Rignovelle
Rigny
Rioz
Roche-et-Raucourt
Roche-sur-Linotte-et-Sorans-les-Cordier
Rochelle 
Ronchamp
Rosey
Rosière 
Rosières-sur-Mance
Roye
Ruhans
Rupt-sur-Saône
St-Barthélemy
St-Bresson
St-Broing
St-Ferjeux
St-Gand
St-Germain
St-Loup-Nantouard
St-Loup-sur-Semouse
St-Marcel
Ste-Marie-en-Chanois
Ste-Marie-en-Chaux
Ste-Reine
St-Remy
St-Sauveur
St-Sulpice
St-Valbert
Saponcourt
Saulnot
Saulx
Sauvigney-lès-Gray
Sauvigney-lès-Pesmes
Savoyeux
Scey-sur-Saône-et-St-Albin
Scye
Secenans
Selles
Semmadon
Senargent-Mignafans
Senoncourt
Servance
Servigney
Seveux
Soing-Cubry-Charentenay
Sorans-lès-Breurey
Sornay
Tartécourt
Tavey
Ternuay-Melay-et-St-Hilaire
Theuley
Thieffrans
Thiénans
Tincey-et-Pontrebeau
Traitiéfontaine
Traves
Tremblois 
Trémoins
Trésilley
Tromarey
Vadans
Vaite
Vaivre 
Vaivre-et-Montoille
Valay
Val-de-Gouhenans 
Vallerois-le-Bois
Vallerois-Lorioz
Val-St-Eloi 
Vandelans
Vanne
Vantoux-et-Longevelle
Varogne
Vars
Va

Mée-sur-Seine 
Meigneux
Meilleray
Melun
Melz-sur-Seine
Méry-sur-Marne
Mesnil-Amelot 
Messy
Misy-sur-Yonne
Mitry-Mory
Moisenay
Moissy-Cramayel
Mondreville
Mons-en-Montois
Montarlot
Montceaux-lès-Meaux
Montceaux-lès-Provins
Montcourt-Fromonville
Montdauphin
Montenils
Montereau-Fault-Yonne
Montereau-sur-le-Jard
Montévrain
Montgé-en-Goële
Monthyon
Montigny-le-Guesdier
Montigny-Lencoup
Montigny-sur-Loing
Montmachoux
Montolivet
Montry
Moret-sur-Loing
Mormant
Mortcerf
Mortery
Mouroux
Mousseaux-lès-Bray
Moussy-le-Neuf
Moussy-le-Vieux
Mouy-sur-Seine
Nandy
Nangis
Nanteau-sur-Essonne
Nanteau-sur-Lunain
Nanteuil-lès-Meaux
Nanteuil-sur-Marne
Nantouillet
Nemours
Chauconin-Neufmontiers
Neufmoutiers-en-Brie
Noisiel
Noisy-Rudignon
Noisy-sur-Ecole
Nonville
Noyen-sur-Seine
Obsonville
Ocquerre
Oissery
Orly-sur-Morin
Ormes-sur-Voulzie 
Ormesson
Othis
Ozoir-la-Ferrière
Ozouer-le-Voulgis
Paley
Pamfou
Paroy
Passy-sur-Seine
Pécy
Penchard
Perthes
Pézarches
Pierre-Levée
Pin 
Plessis-aux-Bois 
Plessis-Feu-Aussoux

Cornimont
Courcelles-sous-Châtenois
Coussey
Crainvilliers
Croix-aux-Mines 
Damas-aux-Bois
Damas-et-Bettegney
Damblain
Darney
Darney-aux-Chênes
Darnieulles
Deinvillers
Denipaire
Derbamont
Destord
Deycimont
Deyvillers
Dignonville
Dinozé
Docelles
Dogneville
Dolaincourt
Dombasle-devant-Darney
Dombasle-en-Xaintois
Dombrot-le-Sec
Dombrot-sur-Vair
Domèvre-sur-Avière
Domèvre-sur-Durbion
Domèvre-sous-Montfort
Domfaing
Domjulien
Dommartin-aux-Bois
Dommartin-lès-Remiremont
Dommartin-lès-Vallois
Dommartin-sur-Vraine
Dompaire
Dompierre
Domptail
Domrémy-la-Pucelle
Domvallier
Doncières
Dounoux
Eloyes
Entre-deux-Eaux
Epinal
Escles
Esley
Essegney
Estrennes
Etival-Clairefontaine
Evaux-et-Ménil
Faucompierre
Fauconcourt
Fays
Ferdrupt
Fignévelle
Fiménil
Florémont
Fomerey
Fontenay
Fontenoy-le-Château
Forge 
Forges 
Fouchécourt
Frain
Fraize
Frapelle
Frebécourt
Fremifontaine
Frenelle-la-Grande
Frenelle-la-Petite
Frénois
Fresse-sur-Moselle
Fréville
Frizon
Gelvécourt-et-Adompt
Gemaingoutte
Gemmelaincourt
Gendre